In [13]:
import geopandas as gpd
import pandas as pd
import os
from shapely.geometry import Point, LineString

# busroute.shp 的 欄位名稱(因每次資料來源提供的會有所不同)
route_routename_col = 'RouteNameZ'
route_direction_col = 'Direction'

# Seq 的欄位名稱 (因每次資料來源提供的會有所不同)
seq_routename_col = 'RouteName'
seq_direction_col = 'Direction'
seq_seq_col = 'Seq'
seq_lat_col = 'Lat'
seq_lng_col = 'Lon'

busroute = gpd.read_file(os.path.join(os.getcwd(),'..', 'input','Shp','BusRoute.shp'))
seq = seq = pd.read_csv(os.path.join(os.getcwd(),'..', 'input','seq.csv'))

In [14]:
# 只做同時有站序 & 路線檔案的
routelist = list(set(list(busroute[route_routename_col])) & set(list(seq[seq_routename_col])))
print("可計算的路線共有:", len(routelist),'條')
only_in_route = list(set(busroute[route_routename_col]) - set(seq[seq_routename_col]))
print("只有路線檔案的路線:", only_in_route)
only_in_seq = list(set(seq[seq_routename_col]) - set(busroute[route_routename_col]))
print("只有站序檔案的路線:", only_in_seq)

可計算的路線共有: 379 條
只有路線檔案的路線: []
只有站序檔案的路線: []


In [15]:
shp_outputfolder = os.path.join(os.getcwd(),'..', 'output','shp')
os.makedirs(shp_outputfolder, exist_ok=True)

以下嘗試

In [30]:
route = routelist[1]
print("現在示範的路線編號為：", route)

directions = [0,1]
direction = directions[0]
print("現在示範的方向為：", direction)

現在示範的路線編號為： 5617
現在示範的方向為： 0


In [31]:
busroute_select = busroute[ (busroute[route_routename_col] == route) & (busroute[route_direction_col] == direction)][[route_routename_col,route_direction_col,'geometry' ]].reset_index(drop = True)
seq_select = seq[ (seq[seq_routename_col] == route) & (seq[seq_direction_col] == direction) ].sort_values(seq_seq_col).reset_index(drop = True)
# 假設 seq_select 包含 Lat 和 Lon 欄位，創建一個 geometry 欄位
seq_select['geometry'] = seq_select.apply(lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1)
# 將 Pandas DataFrame 轉換為 GeoDataFrame
seq_select = gpd.GeoDataFrame(seq_select, geometry='geometry').drop_duplicates(subset=[seq_seq_col]).reset_index(drop = True)
seq_select = seq_select.set_crs(epsg=4326, inplace=True)

In [32]:
busroute_select.to_file(os.path.join(shp_outputfolder, 'busroute_select.shp'))
seq_select.to_file(os.path.join(shp_outputfolder, 'seq_select.shp'))

In [33]:


def snap_points_to_line(stops_gdf, routes_gdf, 
                        route_routename_col, route_direction_col, 
                        seq_routename_col, seq_direction_col, 
                        seq_lat_col, seq_lng_col):
    """
    將公車站點 (stops_gdf) 投影到公車路線 (routes_gdf) 上，並動態帶入欄位名稱。

    Parameters:
        stops_gdf (GeoDataFrame): 包含公車站點的 GeoDataFrame。
        routes_gdf (GeoDataFrame): 包含公車路線的 GeoDataFrame。
        route_routename_col (str): 路線名稱欄位名稱。
        route_direction_col (str): 路線方向欄位名稱。
        seq_routename_col (str): 站點路線名稱欄位名稱。
        seq_direction_col (str): 站點方向欄位名稱。
        seq_seq_col (str): 站點順序欄位名稱。
        seq_lat_col (str): 站點緯度欄位名稱。
        seq_lng_col (str): 站點經度欄位名稱。

    Returns:
        GeoDataFrame: 更新後的公車站點 GeoDataFrame，其中 geometry 已投影到路線。
    """
    import geopandas as gpd
    from shapely.geometry import Point
    snapped_points = []

    for _, stop in stops_gdf.iterrows():
        # 找到與站點路線名稱和方向相符的路線
        matching_route = routes_gdf[(routes_gdf[route_routename_col] == stop[seq_routename_col]) & 
                                    (routes_gdf[route_direction_col] == stop[seq_direction_col])]

        if not matching_route.empty:
            # 取出該路線的 geometry
            line = matching_route.iloc[0].geometry
            # 計算站點投影到該路線的最近點
            snapped_point = line.interpolate(line.project(stop.geometry))
            snapped_points.append(snapped_point)
        else:
            # 如果沒有匹配的路線，保持原點
            snapped_points.append(stop.geometry)

    # 更新站點的 geometry
    stops_gdf = stops_gdf.copy()
    stops_gdf['geometry'] = snapped_points
    seq_select_snapped[seq_lat_col] = seq_select_snapped.geometry.y
    seq_select_snapped[seq_lng_col] = seq_select_snapped.geometry.x



    return stops_gdf

# 假設 busroute_select 和 seq_select 已經是 GeoDataFrames，並且你已經知道欄位名稱
seq_select_snapped = snap_points_to_line(seq_select, busroute_select, 
                                         route_routename_col='RouteNameZ', 
                                         route_direction_col='Direction', 
                                         seq_routename_col='RouteName', 
                                         seq_direction_col='Direction', 
                                         seq_lat_col='Lat', 
                                         seq_lng_col='Lon')

# 保存處理後的數據到新檔案
seq_select_snapped.to_file(os.path.join(shp_outputfolder, 'snapped_stops.shp'))

print("公車站點已成功投影到公車路線上！")


公車站點已成功投影到公車路線上！


In [27]:
seq_select.head(3)

,RouteName,Direction,StopUID,StopName,Seq,Lat,Lon,geometry
0,L306A,0,TAO17083,蘆竹區公所,1,25.046691,121.294695,POINT (121.2947 25.04669)
1,L306A,0,TAO17084,南崁,2,25.047697,121.291986,POINT (121.29199 25.0477)
2,L306A,0,TAO17085,南崁中正,3,25.050373,121.290496,POINT (121.2905 25.05037)


In [28]:
seq_select_snapped['Lat'] = seq_select_snapped.x

AttributeError: 'GeoDataFrame' object has no attribute 'x'

In [22]:
import geopandas as gpd
from shapely.geometry import Point

# 假設 busroute_select 和 seq_select 已經是 GeoDataFrames
# 將 seq_select 的站點投影到 busroute_select 的路線上

def snap_points_to_line(stops_gdf, routes_gdf):
    """
    將公車站點 (stops_gdf) 投影到公車路線 (routes_gdf) 上。

    Parameters:
        stops_gdf (GeoDataFrame): 包含公車站點的 GeoDataFrame。
        routes_gdf (GeoDataFrame): 包含公車路線的 GeoDataFrame。

    Returns:
        GeoDataFrame: 更新後的公車站點 GeoDataFrame，其中 geometry 已投影到路線。
    """
    snapped_points = []

    for _, stop in stops_gdf.iterrows():
        # 找到與站點 RouteName 和 Direction 相符的路線
        matching_route = routes_gdf[(routes_gdf['RouteNameZ'] == stop['RouteName']) & 
                                    (routes_gdf['Direction'] == stop['Direction'])]
        if not matching_route.empty:
            # 取出該路線的 geometry
            line = matching_route.iloc[0].geometry
            # 計算站點投影到該路線的最近點
            snapped_point = line.interpolate(line.project(stop.geometry))
            snapped_points.append(snapped_point)
        else:
            # 如果沒有匹配的路線，保持原點
            snapped_points.append(stop.geometry)

    # 更新站點的 geometry
    stops_gdf = stops_gdf.copy()
    stops_gdf['geometry'] = snapped_points

    return stops_gdf

# 使用該函式處理公車站點
seq_select_snapped = snap_points_to_line(seq_select, busroute_select)

# # 保存處理後的數據到新檔案
seq_select_snapped.to_file(os.path.join(shp_outputfolder, 'snapped_stops.shp'))

print("公車站點已成功投影到公車路線上！")


公車站點已成功投影到公車路線上！


In [21]:
seq_select_snapped

,RouteName,Direction,StopUID,StopName,Seq,Lat,Lon,geometry
0,L306A,0,TAO17083,蘆竹區公所,1,25.046691,121.294695,POINT (121.29465 25.04665)
1,L306A,0,TAO17084,南崁,2,25.047697,121.291986,POINT (121.292 25.04771)
2,L306A,0,TAO17085,南崁中正,3,25.050373,121.290496,POINT (121.29048 25.05037)
3,L306A,0,TAO17086,台茂,4,25.052426,121.286954,POINT (121.28694 25.0524)
4,L306A,0,TAO17087,下南崁,5,25.053404,121.285187,POINT (121.28518 25.05339)
5,L306A,0,TAO17089,南亞(好市多),6,25.055160,121.282224,POINT (121.28222 25.05516)
6,L306A,0,TAO17090,溪州,7,25.057128,121.278975,POINT (121.27897 25.05711)
7,L306A,0,TAO17091,義美,8,25.060221,121.279094,POINT (121.27907 25.06023)
8,L306A,0,TAO17092,戶政地政衛生所,9,25.062901,121.280919,POINT (121.28091 25.0629)
9,L306A,0,TAO17093,義美,10,25.060276,121.279123,POINT (121.2791 25.06029)


In [7]:
# import geopandas as gpd
# from shapely.geometry import LineString, Point
# from shapely.ops import split

# # 假設 busroute_select 和 seq_select 是已經讀入的 GeoDataFrames
# # 確保兩者的 CRS 都是 WGS84 (EPSG:4326)
# busroute_select = busroute_select.to_crs(epsg=4326)
# seq_select = seq_select.to_crs(epsg=4326)

# # 1. 將公車站點吸附到最近的公車路線上
# # 將每個站點吸附到最近的路線
# seq_select["projected_point"] = seq_select.geometry.apply(
#     lambda point: busroute_select.geometry.iloc[0].interpolate(
#         busroute_select.geometry.iloc[0].project(point)
#     )
# )

# # 檢查投影後的點是否有順序問題
# seq_select = seq_select.sort_values("Seq").reset_index(drop=True)

# # 2. 依據吸附後的站點切割公車路線
# # 將吸附後的點轉為一條 MULTIPOINT
# multipoint = seq_select["projected_point"].unary_union

# # 確認 multipoint 是否包含所有預期的點
# if not multipoint or len(seq_select) != len(seq_select["projected_point"].unique()):
#     raise ValueError("投影點數量與站點數量不匹配，請檢查數據！")

# # 將路線切割為多段
# route_line = busroute_select.geometry.iloc[0]  # 假設只有一條路線
# split_result = split(route_line, multipoint)

# # 確保處理 GeometryCollection 的情況
# split_lines = []
# if isinstance(split_result, LineString):
#     split_lines = [split_result]
# elif hasattr(split_result, "geoms"):
#     split_lines = [segment for segment in split_result.geoms if isinstance(segment, LineString)]

# # 檢查切割結果是否與站點數量一致
# if len(split_lines) != len(seq_select) - 1:
#     raise ValueError("切割後的線段數量與站點數量不匹配，請檢查切割結果！")

# # 3. 構建結果 GeoDataFrame
# output_data = []
# for i in range(len(split_lines)):
#     start_seq = seq_select.iloc[i].Seq
#     end_seq = seq_select.iloc[i + 1].Seq if i + 1 < len(seq_select) else None
#     output_data.append({
#         "RouteName": busroute_select.RouteNameZ.iloc[0],
#         "StartSeq": start_seq,
#         "EndSeq": end_seq,
#         "geometry": split_lines[i]
#     })

# output_gdf = gpd.GeoDataFrame(output_data, crs="EPSG:4326")

# # 顯示或保存結果
# print(output_gdf)
# # output_gdf.to_file("split_bus_routes.shp")


C:\Users\kjchang\AppData\Local\Temp\ipykernel_28552\4078657849.py:23: DeprecationWarning: The 'unary_union' attribute is deprecated, use the 'union_all()' method instead.
  multipoint = seq_select["projected_point"].unary_union


ValueError: 投影點數量與站點數量不匹配，請檢查數據！

In [8]:
output_gdf

NameError: name 'output_gdf' is not defined

In [ ]:
# 概念正確但失敗 
# all_segments = []

# for route in routelist[:5]:  # 限制為前 3 條路線進行測試
#     for direction in [0, 1]:  # 方向 0 和 1
#         try:
#             # 選取特定路線和方向的資料
#             busroute_select = busroute[
#                 (busroute[route_routename_col] == route) & 
#                 (busroute[route_direction_col] == direction)
#             ][[route_routename_col, route_direction_col, 'geometry']].reset_index(drop=True)
            
#             seq_select = seq[
#                 (seq[seq_routename_col] == route) & 
#                 (seq[seq_direction_col] == direction)
#             ].sort_values(seq_seq_col).reset_index(drop=True)
            
#             # 將站點轉換為點的 geometry 格式
#             seq_select['geometry'] = seq_select.apply(
#                 lambda row: Point(row[seq_lng_col], row[seq_lat_col]), axis=1
#             )
            
#             # 使用函數計算分段路線
#             gdf_segments = get_busroute_segment(busroute_select, seq_select, seq_seq_col=seq_seq_col)
            
#             # 加入路線名稱與方向欄位
#             gdf_segments['RouteName'] = route
#             gdf_segments['Direction'] = direction
            
#             # 將分段結果加入 all_segments 列表
#             all_segments.append(gdf_segments)
        
#         except:
#             pass

# # 將所有分段結果合併成一個 GeoDataFrame
# all_segments_gdf = gpd.GeoDataFrame(pd.concat(all_segments, ignore_index=True), crs="EPSG:4326")
# all_segments_gdf.to_file(os.path.join(shp_outputfolder, 'segment_route.shp'))
